In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
from pyomo.environ import * 
from pyomo.opt import SolverFactory
import pandas as pd

### Decision Variable:

Let x = Revenue take per trip (in dollars)

### Objective Function:

Maximize the net revenue over a 12-month period on this route.

Revenue = prevailing_rate - prevailing_wage


### Maximize:

z = adjusted revenue

z = gross revenue - lost cost

z = SUMMATION *m* [1,12], (completed rides in month *m* * x) - (CAC churned drivers + CAC churned non-failed riders + CAC churned failed riders)

### Subject to:
* Completed monthly rides = 100 * total number of drivers
* Match Rate = 93 (Based on the initial data)
* max driver churn rate at 5% given wage at $19
* Rider churn who never experienced failed request: 10%
* Rider churn who experienced failed request: 33%

### Constraints:

* x <= 6 (Based on polynomial match rate regression)
* x + prevailing wage <= 25 (Cannot charge riders more than the prevailing rate)


### Crucial information not provided:
* average monthly ratio of drivers:riders
    * we assume 1:20 driver:riders monthly (https://www.earnestanalytics.com/behind-the-ridesharing-wheel/)
* unique drivers added per month
    * ~1% new drivers added monthly assumed from NYC Jan22-Mar23 data (https://toddwschneider.com/dashboards/nyc-taxi-ridehailing-uber-lyft-data/)
* monthly ratio of riders failed_to_find_driver_at_least_once:didnt_fail_to_find_driver

### What the model fails to account for:
* periodicity trends: holidays, seasonality
* rush hour surge pricing and behavior

### Imputations
* Polynomial trend for **match rate given revenue take** to account for both increases and decreases in revenue over time. 2nd Degree Polynomial trend overfits the current data but is used as an assumption for now
* Exponential trend for **driver churn rate based on wave** to account for churn rate never hitting 0% or 100%

### Other Assumptions
* Average Revenue per User (Driver) = Revenue * Match Rate
* LTV = ARPU / Churn Rate
* Maintain a LTV:CAC between 2-3



In [18]:
#read in base calculations to simplify objective function
df = pd.read_excel(r'C:\Users\alxra\Documents\GitHub\Misc_Operations_Research\LyftOptimization.xlsx', 
              sheet_name='python_data', index_col=0)
df = df.round()

df

,cnt_total_drivers,cnt_completed_ride_reqs,cnt_total_ride_reqs,cnt_driver_churn
month,,,,
1,300.0,30000.0,32258.0,15.0
2,285.0,28500.0,30645.0,14.0
3,271.0,27075.0,29113.0,14.0
4,257.0,25721.0,27657.0,13.0
5,244.0,24435.0,26274.0,12.0
6,232.0,23213.0,24961.0,12.0
7,221.0,22053.0,23713.0,11.0
8,210.0,20950.0,22527.0,10.0
9,199.0,19903.0,21401.0,10.0


In [19]:
#init params
driver_churn = 0.05
match_rate = 0.93
rider_churn1 = 0.10
rider_churn2 = 0.33

In [20]:
model = ConcreteModel(name="base")

model.x = Var(domain = NonNegativeReals, name = 'unit take revenue') 

In [23]:
#objective function
## gross revenue - lost cost
z = (
    #month1
    ((df.at[1,'cnt_completed_ride_reqs'] * model.x) - (
                                                        (500 * df.at[1,'cnt_total_drivers'] * driver_churn) +
                                                        (15 * df.at[1,'cnt_completed_ride_reqs'] * rider_churn1) +
                                                        (15 * (df.at[1,'cnt_total_ride_reqs'] 
                                                                   - df.at[1,'cnt_completed_ride_reqs']) * rider_churn2)
    )) +
    #month2
    ((df.at[2,'cnt_completed_ride_reqs'] * model.x) - (
                                                        (500 * df.at[2,'cnt_total_drivers'] * driver_churn) +
                                                        (15 * df.at[2,'cnt_completed_ride_reqs'] * rider_churn1) +
                                                        (15 * (df.at[2,'cnt_total_ride_reqs'] 
                                                                   - df.at[2,'cnt_completed_ride_reqs']) * rider_churn2)
    )) +
    #month3
    ((df.at[3,'cnt_completed_ride_reqs'] * model.x) - (
                                                        (500 * df.at[3,'cnt_total_drivers'] * driver_churn) +
                                                        (15 * df.at[3,'cnt_completed_ride_reqs'] * rider_churn1) +
                                                        (15 * (df.at[3,'cnt_total_ride_reqs'] 
                                                                   - df.at[3,'cnt_completed_ride_reqs']) * rider_churn2)
    )) +
    #month4
    ((df.at[4,'cnt_completed_ride_reqs'] * model.x) - (
                                                        (500 * df.at[4,'cnt_total_drivers'] * driver_churn) +
                                                        (15 * df.at[4,'cnt_completed_ride_reqs'] * rider_churn1) +
                                                        (15 * (df.at[4,'cnt_total_ride_reqs'] 
                                                                   - df.at[4,'cnt_completed_ride_reqs']) * rider_churn2)
    )) +
    #month5
    ((df.at[5,'cnt_completed_ride_reqs'] * model.x) - (
                                                        (500 * df.at[5,'cnt_total_drivers'] * driver_churn) +
                                                        (15 * df.at[5,'cnt_completed_ride_reqs'] * rider_churn1) +
                                                        (15 * (df.at[5,'cnt_total_ride_reqs'] 
                                                                   - df.at[5,'cnt_completed_ride_reqs']) * rider_churn2)
    )) +
    #month6
    ((df.at[6,'cnt_completed_ride_reqs'] * model.x) - (
                                                        (500 * df.at[6,'cnt_total_drivers'] * driver_churn) +
                                                        (15 * df.at[6,'cnt_completed_ride_reqs'] * rider_churn1) +
                                                        (15 * (df.at[6,'cnt_total_ride_reqs'] 
                                                                   - df.at[6,'cnt_completed_ride_reqs']) * rider_churn2)
    )) +
    #month7
    ((df.at[7,'cnt_completed_ride_reqs'] * model.x) - (
                                                        (500 * df.at[7,'cnt_total_drivers'] * driver_churn) +
                                                        (15 * df.at[7,'cnt_completed_ride_reqs'] * rider_churn1) +
                                                        (15 * (df.at[7,'cnt_total_ride_reqs'] 
                                                                   - df.at[7,'cnt_completed_ride_reqs']) * rider_churn2)
    )) +
    #month8
    ((df.at[8,'cnt_completed_ride_reqs'] * model.x) - (
                                                        (500 * df.at[8,'cnt_total_drivers'] * driver_churn) +
                                                        (15 * df.at[8,'cnt_completed_ride_reqs'] * rider_churn1) +
                                                        (15 * (df.at[8,'cnt_total_ride_reqs'] 
                                                                   - df.at[8,'cnt_completed_ride_reqs']) * rider_churn2)
    )) +
    #month9
    ((df.at[9,'cnt_completed_ride_reqs'] * model.x) - (
                                                        (500 * df.at[9,'cnt_total_drivers'] * driver_churn) +
                                                        (15 * df.at[9,'cnt_completed_ride_reqs'] * rider_churn1) +
                                                        (15 * (df.at[9,'cnt_total_ride_reqs'] 
                                                                   - df.at[9,'cnt_completed_ride_reqs']) * rider_churn2)
    )) +
    #month10
    ((df.at[10,'cnt_completed_ride_reqs'] * model.x) - (
                                                        (500 * df.at[10,'cnt_total_drivers'] * driver_churn) +
                                                        (15 * df.at[10,'cnt_completed_ride_reqs'] * rider_churn1) +
                                                        (15 * (df.at[10,'cnt_total_ride_reqs'] 
                                                                   - df.at[10,'cnt_completed_ride_reqs']) * rider_churn2)
    )) +
    #month11
    ((df.at[11,'cnt_completed_ride_reqs'] * model.x) - (
                                                        (500 * df.at[11,'cnt_total_drivers'] * driver_churn) +
                                                        (15 * df.at[11,'cnt_completed_ride_reqs'] * rider_churn1) +
                                                        (15 * (df.at[11,'cnt_total_ride_reqs'] 
                                                                   - df.at[11,'cnt_completed_ride_reqs']) * rider_churn2)
    )) +
    #month12
    ((df.at[12,'cnt_completed_ride_reqs'] * model.x) - (
                                                        (500 * df.at[12,'cnt_total_drivers'] * driver_churn) +
                                                        (15 * df.at[12,'cnt_completed_ride_reqs'] * rider_churn1) +
                                                        (15 * (df.at[12,'cnt_total_ride_reqs'] 
                                                                   - df.at[12,'cnt_completed_ride_reqs']) * rider_churn2)
    ))
    

)

In [27]:
model.objective = Objective(expr = z, sense = maximize)

In [28]:
model.constraints = ConstraintList()
model.constraints.add(model.x <= 6)
model.constraints.add(model.x + 19 <= 25)


In [ ]:
opt = SolverFactory('cbc')
opt.solve(model) 